# Exemplo 03: Análise de Enlace
## Identificação de Aeroportos com mais conectividade

Muitos problemas em ciência de dados podem ser modelados com um grafo. Um exemplo é a análise da rede aérea em uma região, onde os vertices são os aeroportos e as areastas são as linhas aéreas. Usando algoritmos de análise de enlace podemos extrair informações como aeroportos mais movimentados e menor caminho entre duas localidades.

Porém essa análise não é tão trivial. A maneira mais simples de determinar o aeroporto mais movimentado é contar o número de voos realizados de e para esta cidade. No entanto, como a maioria das companhias aéreas utiliza um sistema de *hub-and-spoke*, a simples contagem dos voos de entrada e saída não transmite a importância do aeroporto para o tráfego aéreo geral. Isso ocorre porque determinados aeroportos centrais podem ser pontos de passagem para os vôos em outros aeroportos e, como resultado, esses aeroportos centrais podem ser considerados mais importantes, mesmo que tenham contagens total de voos seja igual ou até menores.

O algoritmo Pagerank foi originalmente criado para medir a importância relativa das páginas da web, avaliando os links de ligação da página. Qualquer página da web é considerada mais importante se outras páginas importantes tiverem links para essa página. Podemos aplicar esse mesmo conceito de importância aos aeroportos. Se você substituir “página da web” por “aeroporto” e substituir “link da web” por “voo da companhia aérea”, poderá ver que o PageRank pode ser usado para avaliar a importância de um aeroporto. O PageRank acaba sendo uma boa maneira de medir a importância do aeroporto, dado o uso do molelo *hub-and-spoke* usado pelas companhias hub e spoke das companhias aéreas. Um aeroporto importante acabaria sendo um aeroporto que por si só é um *hub* no qual outros aeroportos possuem muitos vôos ou um “hub de hubs”.

Para este exemplo vamos usar uma base de dados de aeroportos e voos nos Estados Unidos e Canada. Então, para identificar os aeroportos mais importantes da região, considerando como um determinado aeroporto influencia os vôos para outros aeroportos, podemos usar o algoritmo Pagerank. Este exemplo mostra os aeroportos com mais voos e os aeroportos mais conectados calculados pelo Pagerank.

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import explode

# Import Graphframes lib
from graphframes import GraphFrame

import time
start_time = time.time()

data_path='./data/'

In [2]:
# Cria Spark Session
sparkSession = SparkSession.builder \
       .master("local[*]") \
       .appName("LinkAnalisysAirlines") \
       .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/15 09:51:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Airports

| Field | Description |
| --------- | :-------------: |
| node_id | Unique identifier for the airport. |
| name | Name of airport or city and state.|
| metro_pop | City/region population.|
| latitude | Airport latitude |
| longitude | Airport longitude |


In [3]:
# Le arquivo de dados Airport para Dataframe Spark

airport = sparkSession.read.format("csv").options(sep=',',header='true',inferschema='true').\
         load(data_path+"reachability-meta.csv.gz")

#Exibe campos da tabela e os tipos de dados
#airport.dtypes
airport.show()

+-------+--------------------+---------+---------+-----------+
|node_id|                name|metro_pop| latitude|  longitude|
+-------+--------------------+---------+---------+-----------+
|      0|      Abbotsford, BC| 133497.0|49.051575|-122.328849|
|      1|        Aberdeen, SD|  40878.0| 45.45909| -98.487324|
|      2|         Abilene, TX| 166416.0|32.449175| -99.741424|
|      3|    Akron/Canton, OH| 701456.0| 40.79781| -81.371567|
|      4|         Alamosa, CO|   9433.0| 37.46818|-105.873599|
|      5|          Albany, GA| 157688.0| 31.58076| -84.155989|
|      6|          Albany, NY| 871478.0|42.651455| -73.755274|
|      7|     Albuquerque, NM| 898642.0| 35.08418|-106.648639|
|      8|      Alexandria, LA| 154505.0|31.312685| -92.445649|
|      9|Allentown/Bethleh...| 824916.0|40.651428| -75.434219|
|     10|        Alliance, NE|   8499.0| 42.09712|-102.871454|
|     11|          Alpena, MI|  29386.0|45.061565| -83.445154|
|     12|         Altoona, PA| 127099.0| 40.50719| -78.

## Flights

| Field | Description |
| --------- | :-------------: |
| FromNodeId | Origin airport (node_id).|
| ToNodeId | Destination airport (node_id).|
| Weight | Distance |


In [4]:
# Le arquivo de dados Linhas Aereas para Dataframe Spark

routes = sparkSession.read.format("csv").options(sep=' ',header='true',inferschema='true').\
          load(data_path+"reachability.txt.gz")

#routes.dtypes
routes.show()

+----------+--------+------+
|FromNodeId|ToNodeId|Weight|
+----------+--------+------+
|        27|       0|  -757|
|        57|       0|   -84|
|        70|       0| -1290|
|        74|       0|  -465|
|        86|       0|  -700|
|        94|       0|  -526|
|       100|       0|  -448|
|       113|       0|   -90|
|       138|       0|  -256|
|       154|       0|  -270|
|       166|       0|  -515|
|       178|       0|  -400|
|       230|       0|  -486|
|       235|       0|  -170|
|       242|       0|  -469|
|       246|       0|  -325|
|       262|       0|  -200|
|       269|       0|  -525|
|       275|       0|  -585|
|       280|       0|  -405|
+----------+--------+------+
only showing top 20 rows



## Building Graph

In [5]:
# Extrair campos relevantes para definir os vertices do grafo
vertice = airport.select("node_id","name")

# Graphframe exige que coluna com identificação do vertice possua nome 'id'
# Troca nome "node_id" por "id"
vertice = vertice.withColumnRenamed("node_id", "id")

#caso precise converter algum campo
#vertice = vertice.withColumn("id", vertice["id"].cast("string"))

airport_name = airport.select("node_id", "name").withColumnRenamed("node_id", "id")

#vertice.dtypes
vertice.show(5)

+---+----------------+
| id|            name|
+---+----------------+
|  0|  Abbotsford, BC|
|  1|    Aberdeen, SD|
|  2|     Abilene, TX|
|  3|Akron/Canton, OH|
|  4|     Alamosa, CO|
+---+----------------+
only showing top 5 rows



In [6]:
# Extrair campos relevantes para definir os arestas do grafo
edge = routes.select("FromNodeId","ToNodeId")

# Graphframe exige que colunas com identificação das arestas possua nome 'src' para origem e 'dst' para destino
# Troca nome "FromNodeId" por "src" e "ToNodeId" por "dst"
edge = edge.withColumnRenamed("FromNodeId", "src") \
           .withColumnRenamed("ToNodeId", "dst")

edge.show(5)

+---+---+
|src|dst|
+---+---+
| 27|  0|
| 57|  0|
| 70|  0|
| 74|  0|
| 86|  0|
+---+---+
only showing top 5 rows



In [7]:
# Create a GraphFrame graph
g = GraphFrame(vertice, edge)

# Print number of vertices and edges
print("Airports: ",g.vertices.count())
print("Flights: ",g.edges.count())

/usr/local/lib/python3.8/dist-packages/pyspark/sql/dataframe.py:148: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


Py4JJavaError: An error occurred while calling o64.loadClass.
: java.lang.ClassNotFoundException: org.graphframes.GraphFramePythonAPI
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:589)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:522)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.

## In-degree of each vertex in the graph

In [ ]:
# Aeroportos com mais chegadas

g.degrees.select("id", "degree").join(airport_name, on=['id'], how='inner').select("name", "degree").orderBy("degree",ascending=False).show()

## Triangle Count for each vertice

In [ ]:
# Count the number of triagles in each vertice

g.triangleCount().select("name", "count").orderBy("count",ascending=False).show(20)

## Page Rank

In [ ]:
# Run PageRank algorithm, and show results.
results = g.pageRank(resetProbability=0.01, maxIter=20)
#results = g.pageRank(resetProbability=0.01, tol=0.001)

results.vertices.select("name", "pagerank").orderBy("pagerank",ascending=False).show()

In [ ]:
sparkSession.stop()
print("--- Execution time: %s seconds ---" % (time.time() - start_time))